In [7]:
import pandas as pd
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")

In [8]:
data = pd.read_csv('scdb-200rfq-30cap-metrics.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])

In [9]:
grouped_data = data.groupby('Tx_id')
tx_id = list(grouped_data.groups.keys())
len(tx_id)

15329

In [10]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])

rfq_count = 0
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'REQUEST_FOR_QUOTE':
        rfq_count += 1   


print(f"#RFQs =  {rfq_count}")

#RFQs =  198


In [11]:
df_test = grouped_data.get_group("b861bfd94f3ac9f2186e074690685f5d890a98b108f7f0ba9d0ed13110a50637")

KeyError: 'b861bfd94f3ac9f2186e074690685f5d890a98b108f7f0ba9d0ed13110a50637'

In [ ]:
df_test

In [ ]:
"REQUEST_FOR_QUOTE" == grouped_data.get_group("b861bfd94f3ac9f2186e074690685f5d890a98b108f7f0ba9d0ed13110a50637")["Tx_type"].to_list()[0]

In [12]:
time_all_committed_rfqs = {}
time_passed_rfqs = []
validated_time = []
accepted_time = []
committed_time = []

rfq_underfit = []
rfq_underfit_dict = {}

rfq_overfit = []
rfq_overfit_dict = {}

for i in range(0, len(tx_id)):
    temp_list = []
    if "REQUEST_FOR_QUOTE" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_rfqs[tx_id[i]] = actual_time
            time_passed_rfqs.append(actual_time)
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
            check_time = df[df['Event'] == 'check_tx']['Time'].to_list()[0]
            accepted = check_time - received_time
            accepted_time.append(accepted)
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
        elif len(temp_list) < 6:
            rfq_underfit.append(temp_list)
            rfq_underfit_dict[tx_id(i)] = temp_list
        elif 7 >= len(temp_list) > 6 :
            df_check = grouped_data.get_group(tx_id[i])
            if len(df_check[df_check['Event'] == 'check_tx']['Time'].to_list()) > 1:
                check_time = max(df_check[df_check['Event'] == 'check_tx']['Time'].to_list())
            else:
                check_time = df_check[df_check['Event'] == 'check_tx']['Time'].to_list()[0] 
            
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_rfqs[tx_id[i]] = actual_time
            time_passed_rfqs.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
           
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
            rfq_overfit.append(temp_list)
            rfq_overfit_dict[tx_id[i]] = temp_list

In [13]:
# Analysis of RFQ Transaction
if len(time_passed_rfqs) < 2:
    time_rfqs.append(time_passed_rfqs[0])
rfq_mean = "{:.2f}".format(stat.mean(time_passed_rfqs))
rfq_median = stat.median(time_passed_rfqs)
rfq_max = max(time_passed_rfqs)
rfq_min = min(time_passed_rfqs)
rfq_std_dev = "{:.2f}".format(stat.mean(time_passed_rfqs))

print(rfq_mean)
print(rfq_median)
print(rfq_max)
print(rfq_min)
print(rfq_std_dev)
print(f"total RFQs = {len(time_passed_rfqs)}")

rfq_dict = {
               'min': rfq_min,
               'max': rfq_max,
               'average': rfq_mean,
                'median' : rfq_median,
               'standard_deviation' : rfq_std_dev,
               '# of txs' : len(time_passed_rfqs)}

241.32
231
384
116
241.32
total RFQs = 197


In [14]:
len(rfq_overfit_dict)

187

In [15]:
rfq_overfit_dict

{'0123d639fc37b4e6cca7f02a9b84050cdf9c59b6a0082d71d9828726de83d7cb': [4545,
  4548,
  4557,
  4623,
  4676,
  4679,
  4761],
 '019d92f05642f76fe5ee6187b69af7e04e87cc1c60f26a5bccdf229bdb973096': [1756,
  1759,
  1764,
  1884,
  1922,
  1930,
  2010],
 '0536c4af7babade2fd096c6d84c72be7dde5114fc4e812d71e3ef7f42923a5c6': [1480,
  1485,
  1491,
  1590,
  1654,
  1661,
  1748],
 '061c0092b131e907f3481bc8f3959e963a707077b6a6d8223810e05a7b9f2aea': [4547,
  4550,
  4556,
  4625,
  4677,
  4683,
  4765],
 '068c008cd261508a1838d8bf009500a3d985b6fb70d82dd104279bb6b38ac25a': [4396,
  4400,
  4405,
  4482,
  4534,
  4542,
  4630],
 '0718a5e15259cf53ddc27ce44812af8b3cfd1fe9caee53e3aeb51aea024b0bf2': [4685,
  4688,
  4694,
  4763,
  4816,
  4823,
  4907],
 '089f0f31f92ab9e6b1b4ebef2da2f6c1eec97b8c51bcc5f86eddc628deb83f03': [3036,
  3039,
  3044,
  3120,
  3173,
  3175,
  3258],
 '0f6cf93d3a57ca8419dc6af05558efe1e1bb589462d5dc3be79d2361e426041f': [3865,
  3868,
  3878,
  3944,
  3998,
  4004,
  4083],


In [16]:
results = pd.DataFrame(rfq_dict, index = ['RFQ'])
results.to_csv('rfq_analysis.csv', sep=',', encoding='utf-8')
results

,min,max,average,median,standard_deviation,# of txs
RFQ,116,384,241.32,231,241.32,197


In [17]:
results.to_csv('rfq_analysis.csv', sep=',', encoding='utf-8')

In [18]:
results

,min,max,average,median,standard_deviation,# of txs
RFQ,116,384,241.32,231,241.32,197
